In [1]:
using NBInclude
@nbinclude("diffSHT_healpix.ipynb")
@nbinclude("utilities.ipynb")

In [3]:
using Turing
using Distributions
using Plots
using LinearAlgebra
using Random
using AdvancedHMC
using LogDensityProblems
using LogDensityProblemsAD
using ProgressMeter
using Base.Threads
using MicroCanonicalHMC
using MuseInference
using LaTeXStrings
using AbstractDifferentiation

┌ Warning: You have Turing 0.29.3 but MuseInference requires a version sem-ver compatible with 0.28. 
│ Install a compatible version and restart Julia if you wish to use the MuseInference interface to Turing.
└ @ MuseInference C:\Users\andre\.julia\packages\MuseInference\jjAab\src\MuseInference.jl:44


## Generate fictional data

In [4]:
@model function Chol_data_generator(l_max, n_tomo)
    L = Vector{Matrix{Float64}}(undef, l_max+1)# a vector of l_max+1 matrices, each of them will 2n_tomo x 2n_tomo elements
    x = Vector{Matrix{Float64}}(undef, l_max+1)# a vector of l_max+1 matrices, the i-th matrix has (2ntomo x i) elements
    
    for i in 1:l_max+1
        L[i] ~ LKJCholesky(2*n_tomo, 1)
        x[i] ~ arraydist([MvNormal(zeros(2*n_tomo), Matrix{Float64}(I,2*n_tomo,2*n_tomo)) for p in -(i-1):(i-1)])
    end
end

Chol_data_generator (generic function with 2 methods)

In [5]:
@model function data_generator(l_max, n_tomo)
    C = Vector{Matrix{Float64}}(undef, l_max+1)
    ã = Vector{Matrix{Float64}}(undef, l_max+1)
    
    for i in 1:l_max+1
        C[i] ~ LKJ(2*n_tomo, 1)
        ã[i] ~ arraydist([MvNormal(zeros(2*n_tomo), C[i]) for p in -(i-1):(i-1)])
    end
end

data_generator (generic function with 2 methods)

Hyperparameters' choice:

In [6]:
nbin = 1
lmax = 31
q = 0#-0.5*(2*nbin+1)
nside = 16
npix = nside2npix(nside)

ε = 0.27/sqrt(2*3)  
N_noise = ε * I
inv_N_noise = inv(N_noise);


We can generate now some fictional data:

In [7]:
gen_data = rand(data_generator(lmax, nbin))
gen_C = collect(gen_data[i] for i in 1:2:2*(lmax+1))
gen_ã = collect(gen_data[i] for i in 2:2:2*(lmax+1));

In [8]:
gen_a = deepcopy(gen_ã)
for i in 1:lmax+1
    gen_a[i][:,2:end] /= sqrt(2)
end

In [9]:
hpix_gen_alms = from_alm_to_healpix_alm(gen_a, lmax, nbin)
gen_maps = Array{HealpixMap}(undef, 2*nbin)
for i in 1:2*nbin
    gen_maps[i] = alm2map(hpix_gen_alms[i], nside)
end

### Generate initial condition

In [10]:
free_param_n = (2*nbin - 1)*(nbin + 1)
num_par = 2*nbin*(lmax+1)*(lmax+1) + (lmax+1)*free_param_n

2112

In [11]:
gen_L = [cholesky(gen_C[i]).L for i in 1:lmax+1];

In [12]:
gen_K = deepcopy(gen_L)
for l in 1:lmax+1
    for i in 1:2*nbin
        gen_K[l][i,i] = log(gen_L[l][i,i])
    end
end

In [13]:
gen_x = inv.(gen_L) .* gen_ã;

In [14]:
x_vec = x_vecmat2vec(gen_x, lmax, nbin)
k_vec = vectorK_vecmat2vec(gen_K, nbin, lmax);

In [15]:
gen_θ = Vector{Float64}(vcat(x_vec, k_vec))
θ₀ = rand(MvNormal(gen_θ, 0.001*I));

## Neg-log-posterior

In Cholesky coordinates, the negative log-posterior to sample looks like
$$  \psi(\mathbf{x},\mathrm{K}) = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x})^{\mathrm{T}}\mathrm{N}^{-1}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x}) + \frac{\mathbf{x}^{\mathrm{T}}\mathbf{x}}{2} + \sum_{\ell}\sum_{\alpha=1}^{n}(\alpha-2-n-2q)\mathrm{K}_{\ell , \alpha\alpha} \right] $$
$$  \psi(\mathbf{x},\mathrm{L}) = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x})^{\mathrm{T}}\mathrm{N}^{-1}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x}) + \frac{1}{2}\sum_{\ell}\sum_{m}\mathbf{x}_{\ell m}^{\mathrm{T}}\mathbf{x}_{\ell m} + \sum_{\ell}\sum_{\alpha=1}^{n}(\alpha-2-n-2q)\ln{\mathrm{L}_{\ell , \alpha\alpha}} \right]$$
The noise matrix $N$, which is in real (pixel) space is taken to be a diagonal matrix of dimension $n_{pix}$.

The index $n$ is $2*n_{bins}$ .

A Jeffreys prior (as said in ALMANAC) corresponds to 
$$ q = -\frac{N_p+1}{2} \ , $$ where $C_\ell$ has $N_p \times N_p$ entries; this means $N_p=2*n_{bins}$

### Neg-log-likelihood

$$-\log{(\mathcal{L}(\mathrm{L},\mathbf{x}))} = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x})^{\mathrm{T}}\mathrm{N}^{-1}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x}) \right]$$

function from_K_to_L(K::Vector{Matrix{Float64}})
    L = deepcopy(K)
    for l in 1:lmax+1
        for i in 1:2*nbin
            L[l][i,i]=exp(K[l][i,i])
        end
    end
    return L
end     

In [16]:
function negloglikelihood(L::Vector{Matrix{Float64}}, x::Vector{Matrix{Float64}}; maps)
    loglike = 0.
    a = Chol_Lx2a(x, L)
    alms = from_alm_to_healpix_alm(a, lmax, nbin)
    for i in 1:2*nbin
        d̃ = alm2map(alms[i], nside)
        loglike += 0.5*transpose(maps[i]-d̃)*inv_N_noise*(maps[i]-d̃)
    end
    return loglike
end

negloglikelihood (generic function with 1 method)

### Neg-log-prior1
$$-\log{(\mathcal{P}_1(\mathbf{x}))} = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}\sum_{\ell}\sum_{m}\mathbf{x}_{\ell m}^{\mathrm{T}}\mathbf{x}_{\ell m} \right] $$

In [17]:
function neglogprior1(x::Vector{Float64})
    logprior =  0.5*sum(x .* x)
    return logprior
end

@adjoint function neglogprior1(x::Vector{Float64})
    y = neglogprior1(x)
    function neglogprior1_pullback(ȳ)
        x̄ = deepcopy(x) .* ȳ
        return (x̄,)
    end
    return y, neglogprior1_pullback
end

### Neg-log-prior2
$$-\log{(\mathcal{P}_2(L))} = \sum_{j=1}^{2n_{bins}} \left[ \sum_{\ell}\sum_{i=1}^{n}(\alpha-2-n-2q){\mathrm{K}_{\ell , ii}} \right] $$

In [18]:
function neglogprior2(K::Vector{Matrix{Float64}})
    c = 2 + (2*nbin) + (2*q)
    logprior = 0.
    for i in 2:2*nbin
        cons = i-c
        for l in 1:lmax+1
            logprior += cons*K[l][i,i]
        end
    end
    return logprior
end

@adjoint function neglogprior2(K::Vector{Matrix{Float64}})
    y = neglogprior2(K)
    function neglogprior2_pullback(ȳ)
        c = 2 + (2*nbin) + (2*q)
        single_K̄ = diagm([i-c for i in 1:2*nbin])
        single_K̄[1,1] = 0.0
        K̄ = [single_K̄ for i in 1:lmax+1]
        return (K̄,)
    end
    return y, neglogprior2_pullback
end

### Neg-log-posterior - custom target

In [19]:
function neglogpost(θ; maps=gen_maps)
    vec_x = θ[1:2*nbin*(2*numberOfAlms(lmax)-(lmax+1))]
    vec_k = θ[2*nbin*(2*numberOfAlms(lmax)-(lmax+1))+1:end]
    K = vectorK_vec2vecmat(vec_k, nbin, lmax, free_param_n)
    x = x_vec2vecmat(vec_x, lmax, nbin)
    L = vector_from_k_to_L(vec_k, nbin, lmax, free_param_n)
    return negloglikelihood(L, x; maps=maps) + neglogprior1(vec_x) + neglogprior2(K)
end

neglogpost (generic function with 1 method)

In [20]:
ψ, dψ = withgradient(θ->neglogpost(θ), θ₀)

(val = 5101.101622287971, grad = ([-45.782005525203644, -6.264308892513235, 131.1196335120835, -69.55440925233925, -168.07680670791956, 146.30839576415022, -128.8582550661904, 74.35545991603793, -50.94145251098945, -42.220516992307914  …  -663.278238369056, 1377.646386937737, -1659.9525423196364, 4610.219854681885, -2528.840899290636, 4533.43442730399, 114.04568909834208, 4695.584217809345, 2196.23278271038, 952.2286423491715],))

In [22]:
@benchmark neglogpost(θ₀)

BenchmarkTools.Trial: 1561 samples with 1 evaluation.
 Range (min … max):  1.833 ms … 10.321 ms  ┊ GC (min … max): 0.00% … 52.82%
 Time  (median):     2.556 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.191 ms ±  1.279 ms  ┊ GC (mean ± σ):  0.88% ±  4.45%

    ▁▄██▆▇▆▃▂            ▁▂▃▄▄▃▃▁▂                           ▁
  ▄▆█████████████▆▅▇▄▆▇▇████████████████▆▇▆▆▄▄▆▄▄▄▁▁▆▁▅▆▅▅▇▅ █
  1.83 ms      Histogram: log(frequency) by time     7.58 ms <

 Memory estimate: 428.16 KiB, allocs estimate: 467.

In [23]:
@benchmark withgradient(θ->neglogpost(θ), θ₀)

BenchmarkTools.Trial: 748 samples with 1 evaluation.
 Range (min … max):  5.476 ms …  14.090 ms  ┊ GC (min … max): 0.00% … 18.70%
 Time  (median):     6.465 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.677 ms ± 889.598 μs  ┊ GC (mean ± σ):  1.05% ±  4.43%

       ▂▃▆███▆▅▃▂▁                                             
  ▄▄▄▇▇███████████▅█▅█▅▆▁▆▅▁▄▁▅▆▅▅▁▁▁▄▁▆▆▆▆▅▅▄▄▅▄▁▁▄▁▁▁▄▁▄▄▁▄ █
  5.48 ms      Histogram: log(frequency) by time      11.2 ms <

 Memory estimate: 1.19 MiB, allocs estimate: 2526.

In [24]:
dim_params = length(gen_θ)

2112

In [25]:
struct LogTargetDensity
    dim::Int
end

LogDensityProblemsAD.logdensity(p::LogTargetDensity, θ) = -neglogpost(θ)
LogDensityProblemsAD.dimension(p::LogTargetDensity) = p.dim
LogDensityProblemsAD.capabilities(::Type{LogTargetDensity}) = LogDensityProblemsAD.LogDensityOrder{0}()

In [45]:
ℓπ = LogTargetDensity(dim_params)

LogTargetDensity(160)

In [26]:
ℓπ = LogTargetDensity(dim_params)
n_LF = 50

metric = DenseEuclideanMetric(dim_params)
hamiltonian = Hamiltonian(metric, ℓπ, Zygote)
initial_ϵ = find_good_stepsize(hamiltonian, θ₀)
integrator = Leapfrog(initial_ϵ)
kernel = HMCKernel(Trajectory{EndPointTS}(integrator, FixedNSteps(n_LF)))
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.9, integrator))

StanHMCAdaptor(
    pc=WelfordCov,
    ssa=NesterovDualAveraging(γ=0.05, t_0=10.0, κ=0.75, δ=0.9, state.ϵ=0.003125),
    init_buffer=75, term_buffer=50, window_size=25,
    state=window(0, 0), window_splits()
)

In [27]:
nchains = 1
sample_chain = Vector{Any}(undef, nchains)
stats_chain = Vector{Any}(undef, nchains);

In [28]:
n_samples, n_adapts = 1000, 10_000

(1000, 10000)

In [ ]:
for i in 1:nchains
    samples, stats = sample(hamiltonian, kernel, θ₀, n_samples, adaptor, n_adapts; progress=true, verbose=false)
    sample_chain[i] = samples
    stats_chain[i] = stats
end

### Neg-log-posterior - Turing model

In [29]:
Turing.setadbackend(:zygote)

:zygote

@model function posterior(data, noise, θ_len, nbin, nside, lmax, free_param_n)
    
    θ ~ MvNormal(zeros(θ_len), I)
    vec_x = θ[1:(θ_len - (lmax+1)*free_param_n)]
    vec_k = θ[(θ_len - (lmax+1)*free_param_n)+1:end]

    K = vectorK_vec2vecmat(vec_k, nbin, lmax, free_param_n)
    L = vector_from_k_to_L(vec_k, nbin, lmax, free_param_n)
    x = x_vec2vecmat(vec_x, lmax, nbin)

    a = Chol_Lx2a(x, L)
    alms = from_alm_to_healpix_alm(a, lmax, nbin)

    for i in 1:2*nbin
        d̃ = alm2map(alms[i], nside)
        data[i] ~ MvNormal(d̃, noise)
    end
end